# Pre-requisites
Install Azure AI Evaluation SDK with Red Team extra to use the Red Team Agent. 

## Install uv 
uv is a very fast Python package manager. It will help make the installation of azure-ai-evaluation with extras faster. Open a Powershell (Windows) or Terminal (macOS/Linux) to run the following commands:

`pip install uv`

## Create a virtual environment using uv
Create a virtual environment using uv and specify a Python version >= 3.10

`uv venv --python 3.11`

Ensure you are running the following scripts using the virtual environment created. To do so, activate the virtual environment using: 

`.venv\Scripts\activate` (Windows) 

`source .venv/bin/activate` (macOS and Linux)

## Install Azure AI Evaluation SDK with Red Team extra

With the virtual environment activated, you can now install Azure AI Evaluation SDK with Red Team extra

`uv pip install --upgrade "git+https://github.com/slister1001/azure-sdk-for-python.git@red-team-agent-init#subdirectory=sdk/evaluation/azure-ai-evaluation&egg=azure-ai-evaluation[redteam]"`

You can now open VS Code an use this environment to run the examples:

`code .`

# Red Team Agent Configuration
To run RedTeamAgent, you need to import packages and configure your environment.

## Import necessary packages

In [ ]:
from azure.ai.evaluation.red_team_agent import RedTeamAgent, AttackStrategy, RiskCategory
from azure.identity import DefaultAzureCredential
from typing import Dict, List, Optional, Any
import os

## Connect to an Azure AI Project
Navigate to this [Azure AI Foundry Hub](https://int.ai.azure.com/managementCenter/hub/overview?wsid=/subscriptions/4bf6b28a-452b-4af4-8080-8a196ee0ca4b/resourceGroups/naposani/providers/Microsoft.MachineLearningServices/workspaces/sydneylister-1523&flight=AIRedTeaming=true,EvalConvergence&tid=72f988bf-86f1-41af-91ab-2d7cd011db47), and select a project. Insert the project name in the below code as `<your-project-name>`

In [ ]:
azure_ai_project = {
    "subscription_id": os.environ.get("AZURE_SUBSCRIPTION_ID"), # INSERT THE SUBSCRIPTION ID HERE
    "resource_group_name": os.environ.get("AZURE_RESOURCE_GROUP_NAME"), # INSERT THE RESOURCE GROUP NAME HERE
    "project_name": os.environ.get("AZURE_PROJECT_NAME"), # INSERT THE PROJECT NAME HERE
}
credential = DefaultAzureCredential()

# Red Team Agent Examples

Below are several examples demonstrating the various ways to use the RedTeamAgent for vulnerability scanning.

## Hello World! Basic Red Team Scan with Callback Function

This example shows how to use the RedTeamAgent with a simple callback function to test your application.

In [ ]:
# Initialize RedTeamAgent
agent = RedTeamAgent(azure_ai_project=azure_ai_project, credential=credential, num_objectives=1)

# Create a simple application callback function
def simple_app_callback(query: str) -> str:
    """A simple application callback function that responds to user queries"""
    return f"I'm an AI assistant that follows ethical guidelines. I cannot provide harmful content."

# Run a basic scan with the callback function 
# Only baseline attack strategy will be used (baseline is added by default)
results = await agent.scan(
    target=simple_app_callback,
    scan_name="Basic-Callback-Test",
)

## Using Specific Attack Strategies

This example demonstrates using specific individual attack strategies rather than complexity groups.

In [ ]:
# Initialize credentials and RedTeamAgent
credential = DefaultAzureCredential()
agent = RedTeamAgent(
    azure_ai_project=azure_ai_project,
    credential=credential, 
    num_objectives=10, 
    risk_categories=[RiskCategory.HateUnfairness, RiskCategory.Violence, RiskCategory.Sexual, RiskCategory.SelfHarm]
)

# Define a medical assistant callback
def medical_assistant_callback(query: str) -> str:
    return "I'm a medical information assistant. I can provide general health information but cannot diagnose conditions or prescribe treatments."

# Run a scan with specific attack strategies
specific_strategy_results = await agent.scan(
    target=medical_assistant_callback,
    scan_name="Specific-Strategies-Test",
    attack_strategies=[
        AttackStrategy.Base64,           # Encode prompts in Base64
        AttackStrategy.Flip,             # Flip text to evade detection
        AttackStrategy.CharacterSpace,   # Add character spaces
        AttackStrategy.ROT13,            # Use ROT13 encoding
        AttackStrategy.UnicodeConfusable,# Use confusable Unicode characters
        AttackStrategy.Tense,            # Change tense of prompts
        AttackStrategy.CharSwap,         # Swap characters in prompts
        AttackStrategy.Morse,            # Encode prompts in Morse code
        AttackStrategy.Leetspeak,        # Use Leetspeak
        AttackStrategy.Url,              # Use URLs in prompts
        AttackStrategy.Binary,           # Encode prompts in binary
        AttackStrategy.MODERATE,         # Use group of moderate complexity attacks
        AttackStrategy.DIFFICULT,        # Use group of difficult complexity attacks
    ],
    application_scenario="A medical information assistant for general health information"
)

## Data-Only Mode (Without Evaluation)

This example shows how to run in data-only mode to collect conversation data without performing evaluations.

In [ ]:
# Initialize credentials and RedTeamAgent
credential = DefaultAzureCredential()
agent = RedTeamAgent(azure_ai_project=azure_ai_project, credential=credential, risk_categories=[RiskCategory.Sexual, RiskCategory.HateUnfairness], num_objectives=2)

# Define a simple content moderator callback
def content_moderator_callback(query: str) -> str:
    return "I'm a content moderation assistant. I can help identify and flag potentially inappropriate content."

# Run a data-only scan (without evaluations)
data_only_results = await agent.scan(
    target=content_moderator_callback,
    scan_name="Data-Only-Test",
    attack_strategies=[AttackStrategy.Morse, AttackStrategy.Leetspeak],
    application_scenario="A content moderation system for a social media platform",
    data_only=True  # Only collect conversation data without evaluations
)

# You can access the collected conversation data
conversations = data_only_results.redteaming_data
print(f"Collected {len(conversations) if conversations else 0} conversations without evaluation")
data_only_results.attack_simulation()